In [2]:
import subprocess
import os

caminho_para_dji_irp = r'C:\caminho\para\dji_irp.exe' # Onde está salvo o diretório do dji_irp.exe no seu computador
caminho_para_imagens = r'C:\caminho\para\imagens' # Pasta com arquivos R-JPEG 
caminho_para_saida = r'sdk/raw' # Pasta para salvar arquivos RAW de saída

# Lista de arquivos no diretório de imagens
lista_de_arquivos = os.listdir(caminho_para_imagens)

# Loop sobre cada arquivo no diretório de imagens
for arquivo in lista_de_arquivos:
    if arquivo.endswith(".JPG") or arquivo.endswith(".jpg"):
        caminho_completo_entrada = os.path.join(caminho_para_imagens, arquivo)
        nome_saida_raw = os.path.splitext(arquivo)[0] + '.raw'
        caminho_completo_saida = os.path.join(caminho_para_saida, nome_saida_raw)

        # Comando para cada imagem
        comando = f'{caminho_para_dji_irp} -a measure -s {caminho_completo_entrada} -o {caminho_completo_saida} --measurefmt float32'

        # Execute o comando no terminal
        subprocess.run(comando, shell=True)

In [3]:
import os
import numpy as np
from osgeo import gdal, gdal_array

caminho_para_pasta_raw = 'caminho/para/pasta/raw' # Pasta com arquivos RAW
caminho_para_pasta_tiff = 'caminho/para/pasta/tiff' # Pasta para salvar arquivos TIFF de saída

# Dimensões da imagem (em pixel)
largura = 640 # Informar largura da imagem
altura = 512 # Informar altura da imagem

# Lista de todos os arquivos .raw na pasta
arquivos_raw = [f for f in os.listdir(caminho_para_pasta_raw) if f.endswith('.raw')]

# Processamento de cada arquivo .raw
for arquivo_raw in arquivos_raw:
    caminho_completo_entrada = os.path.join(caminho_para_pasta_raw, arquivo_raw)

    # Geração do nome do arquivo de saída .tiff
    nome_saida_tiff = os.path.splitext(arquivo_raw)[0] + '.tiff'
    caminho_completo_saida = os.path.join(caminho_para_pasta_tiff, nome_saida_tiff)

    # Carregamento dos dados brutos do arquivo .raw
    with open(caminho_completo_entrada, 'rb') as file:
        raw_data = np.fromfile(file, dtype=np.float32).reshape((altura, largura))

    # Salva a imagem TIFF usando a biblioteca GDAL
    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(caminho_completo_saida, largura, altura, 1, gdal.GDT_Float32)
    dataset.GetRasterBand(1).WriteArray(raw_data)
    dataset.FlushCache()
    dataset = None  # Fecha o datasetD

In [ ]:
import os
import subprocess

caminho_da_pasta_jpg = 'caminho/para/pasta/raw' # Pasta com arquivos R-JPEG
caminho_da_pasta_tiff = 'caminho/para/pasta/tiff' # Pasta para salvar arquivos TIFF de saída

for arquivo_jpg in os.listdir(caminho_da_pasta_jpg):
    if arquivo_jpg.endswith('.JPG'):
        caminho_completo_jpg = os.path.join(caminho_da_pasta_jpg, arquivo_jpg)

        # Verifica se existe um arquivo TIFF correspondente
        arquivo_tiff = os.path.join(caminho_da_pasta_tiff, f"{os.path.splitext(arquivo_jpg)[0]}.tiff")
        if os.path.exists(arquivo_tiff):
            comando = ['exiftool', '-tagsFromFile', caminho_completo_jpg, '-overwrite_original', arquivo_tiff]
            subprocess.run(comando)

In [ ]:
import tifffile
from pyproj import Transformer
import glob
import os
from osgeo import gdal, osr
from affine import Affine
import math
from xml.etree impor#t ElementTree as ET

# Caminho para a pasta contendo os arquivos GEOTIFF rotacionados
output_rotated_folder = 'caminho/para/pasta/geotiff_rotated' # Pasta para salvar arquivos GEOTIFF rotacionados de saída 
os.makedirs(output_rotated_folder, exist_ok=True)

def raster_center(raster):
    width, height = raster.RasterXSize, raster.RasterYSize
    xmed = width / 2
    ymed = height / 2
    return (xmed, ymed)

def rotate_gt(affine_matrix, angle, pivot=None):
    affine_src = Affine.from_gdal(*affine_matrix)
    affine_dst = affine_src * affine_src.rotation(angle, pivot)
    return affine_dst.to_gdal()

Tam_Pixel = 0.1125 # Informar o tamanho do píxel (calculado com base na altura relativa e FOV)
    
# Caminho para a pasta contendo os arquivos TIFF
pasta_tiffs = 'caminho/para/pasta/tiff' # Pasta com arquivos TIFF

# Padrão para arquivos TIFF
padrao_tiff = '*.tiff'

# Caminho para a pasta contendo os arquivos GEOTIFF
output_geotif_folder = 'caminho/para/pasta/geotiff' # Pasta para salvar arquivos GEOTIFF de saída

# Obter lista de arquivos TIFF na pasta
caminhos_tiffs = glob.glob(f'{pasta_tiffs}{padrao_tiff}')

# Iterar sobre todos os arquivos TIFF na pasta
for caminho_arquivo_tiff in caminhos_tiffs:
    print(f'Processando arquivo: {caminho_arquivo_tiff}')

    # Abrir o arquivo TIFF com o tifffile
    with tifffile.TiffFile(caminho_arquivo_tiff) as tiff:
        # Iterar sobre todas as tags e seus valores
        for tag in tiff.pages[0].tags.values():
            # Verificar se a tag é GPSTag
            if tag.name == 'GPSTag':
                # Extrair as informações de latitude e longitude do dicionário GPSTag
                latitude = tag.value['GPSLatitude']
                longitude = tag.value['GPSLongitude']

                # Converter as partes inteiras e fracionárias para valores decimais
                latitude_decimal = latitude[0] + latitude[2] / 60 + (latitude[4] / (latitude[5] * 3600))
                longitude_decimal = longitude[0] + longitude[2] / 60 + (longitude[4] / (longitude[5] * 3600))

                # Ajustar a latitude e longitude com base na referência (N, S, E, W)
                if tag.value['GPSLatitudeRef'] == 'S':
                    latitude_decimal = -latitude_decimal

                if tag.value['GPSLongitudeRef'] == 'W':
                    longitude_decimal = -longitude_decimal

                # 2ª PARTE - TRANSFORMAR COORDENADAS DE WGS PARA SIRGAS 2000 / UTM zone 22S
                wgs84_epsg = 4326  # WGS84
                sirgas_utm_epsg = 31982  # SIRGAS 2000 / UTM zone 22S
                transformer = Transformer.from_crs(wgs84_epsg, sirgas_utm_epsg)
                long_proj, lat_proj = transformer.transform(latitude_decimal, longitude_decimal)
                print(f'Latitude projetada: {lat_proj}, Longitude projetada: {long_proj}')
                print('-' * 30)  # Linha separadora entre os resultados de diferentes arquivos
        
        # Extraindo o bloco XMP dos dados EXIF
        xmp_data = tiff.pages[0].tags['XMP'].value

        # Analisando os dados XMP para encontrar a RelativeAltitude e GimbalYawDegree
        root = ET.fromstring(xmp_data)
        ns = {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'drone-dji': 'http://www.dji.com/drone-dji/1.0/'}

        # Encontrando o elemento que contém o RelativeAltitude
        relative_altitude_element = root.find('.//rdf:Description/drone-dji:RelativeAltitude', namespaces=ns)

        if relative_altitude_element is not None:
            # Obtendo o valor da RelativeAltitude
            relative_altitude_value = float(relative_altitude_element.text)
            print(f'RelativeAltitude: {relative_altitude_value}')

        # Encontrando o elemento que contém o GimbalYawDegree
        gimbal_yaw_degree_element = root.find('.//rdf:Description/drone-dji:GimbalYawDegree', namespaces=ns)

        if gimbal_yaw_degree_element is not None:
            # Obtendo o valor do GimbalYawDegree
            gimbal_yaw_degree_value = float(gimbal_yaw_degree_element.text)
            print(f'GimbalYawDegree: {gimbal_yaw_degree_value}')
            
        # Encontrando o elemento que contém o GimbalYawDegree
        FlightYawDegree_element = root.find('.//rdf:Description/drone-dji:FlightYawDegree', namespaces=ns)

        if FlightYawDegree_element is not None:
            # Obtendo o valor do FlightYawDegree_element
            FlightYawDegree_value = float(FlightYawDegree_element.text)
            print(f'FlightYawDegree: {FlightYawDegree_value}')

        # 4ª PARTE - TRANSFORMAR O TIFF EM GEOTIFF
        dst_filename = os.path.join(output_geotif_folder, f"{os.path.basename(caminho_arquivo_tiff)}.tiff")
        src_ds = gdal.Open(caminho_arquivo_tiff)
        format = "GTiff"
        driver = gdal.GetDriverByName(format)

        # Obter informações sobre a imagem original
        width = src_ds.RasterXSize
        height = src_ds.RasterYSize
        print(width)
        
        #largura = int(src_ds.RasterXSize*Tam_Pixel)
        #altura = int(src_ds.RasterYSize*Tam_Pixel)

        # Calcular as coordenadas do canto superior esquerdo
        ulx = long_proj - (0.5 * width * Tam_Pixel)  # Considere metade da largura
        uly = lat_proj + (0.5 * height * Tam_Pixel)  # Considere metade da altura
        print(ulx, uly)

        # Configurar a transformação geográfica
        gt = [ulx, Tam_Pixel, 0, uly, 0, -Tam_Pixel]

        # Criar o conjunto de dados de destino
        dst_ds = driver.Create(dst_filename, width, height, 1, gdal.GDT_Float32)

        # Adicionar informações de projeção
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(31982)  # SIRGAS 2000 / UTM zone 22S
        dst_ds.SetProjection(srs.ExportToWkt())

        # Configurar informações de coordenadas georreferenciadas
        dst_ds.SetGeoTransform(gt)

        # Ler os dados da imagem original e gravar no destino
        data = src_ds.GetRasterBand(1).ReadAsArray()
        dst_ds.GetRasterBand(1).WriteArray(data)

        # Construir o caminho para o arquivo rotacionado
        caminho_arquivo_rotacionado = os.path.join(output_rotated_folder, f"rotated_{os.path.basename(caminho_arquivo_tiff)}.tiff")

        # Criar uma cópia do arquivo TIFF original
        dataset_dst = driver.CreateCopy(caminho_arquivo_rotacionado, dst_ds, strict=0)
        center = raster_center(src_ds)
        
        # Calcular a matriz afim rotacionada
        rotated_affine = rotate_gt(gt, FlightYawDegree_value, center)

        # Definir a transformação geográfica do arquivo TIFF rotacionado
        dataset_dst.SetGeoTransform(rotated_affine)

        # Fechar os datasets
        src_ds = None
        dst_ds = None
        dataset_dst = None
    
print('Processamento concluído.')

In [ ]:
from osgeo import gdal

output_rotated_folder = 'caminho/para/pasta/geotiff_rotated' # Pasta com arquivos para realização do mosaico
search_criteria = "r*.tiff"
q = os.path.join(dirpath, search_criteria)
files_to_mosaic = glob.glob(q)
files_to_mosaic.reverse
g = gdal.Warp("caminho/para/pasta/mosaico.tif", files_to_mosaic, format="GTiff", # Pasta onde será salvo o mosaico e nome do mosaico.tif
              options=["COMPRESS=LZW", "TILED=YES"])
g = None 